In [1]:
import numpy as np
import pandas as pd

Я работала в гугл колабе, при необходимости поменяйте две ячейки внизу на свой способ хранения данных.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = '/content/drive/MyDrive/Recsys_data1.csv'

In [4]:
df = pd.read_csv(path)

В анализе удалим то, что нам не нужно по задаче и сделаем обычные проверки данных: содержат ли данные nan и как они распределены по дням.

Также сделаем несколько специфических проверок - проверим, чтобы g0 и g1 были положительными и узнаем, есть ли 0 в impressions.

In [7]:
def analysis(data: pd.DataFrame):
    data = data.drop(['campaign_clicks'], axis=1) #remove unnecessary cols (according to the task)
    print(data.head)

    print("Data contains null is ", data.isnull().values.any()) #check if contains null

    print(len(df[df['impressions'] == 0])) #check if impressions contains 0
    
    #count how many data on each day we have
    cur = data.copy()
    cur['date_time'] = cur['date_time'].apply(lambda x: x[:10])
    cur = cur.groupby(['date_time']).size()
    print(cur)

    print("How many g0 are less than zero: ", len(data[data['g0'] < 0]))
    print("How many g1 are less than zero: ", len(data[data['g1'] < 0]))
    return data

In [8]:
df = analysis(df)

<bound method NDFrame.head of                            date_time  zone_id  banner_id            oaid_hash  \
0         2021-09-27 00:01:30.000000        0          0  5664530014561852622   
1         2021-09-26 22:54:49.000000        1          1  5186611064559013950   
2         2021-09-26 23:57:20.000000        2          2  2215519569292448030   
3         2021-09-27 00:04:30.000000        3          3  6262169206735077204   
4         2021-09-27 00:06:21.000000        4          4  4778985830203613115   
...                              ...      ...        ...                  ...   
15821467  2021-10-02 15:51:35.000000      146        530  4329496688011613719   
15821468  2021-09-27 22:03:14.000000       12         22   453968700792456599   
15821469  2021-10-02 17:41:10.000000       12       1236  9112780675655118328   
15821470  2021-09-29 00:39:32.000000      967         21  6968514095695555037   
15821471  2021-09-28 07:00:18.000000       19        635  8754492963501134426  

Выводы из анализа данных:
1. Есть ненужная числовая зависимость в колонках zone_id, banner_id, os_id, country_id, значит в обработке будем использовать например OneHotEncoding.

2. В данных есть null. У нас нет никаких указаний, что делать с такими данными, так что просто их удалим в обработке.

3. Все impressions равны 1 (что логично), так что уберем этот столбец.

4. Также есть день 2021-09-01, для которого есть всего одна запись и он далеко от остальных. Видимо он попал в данные случайно, его надо удалить.

5. Последний день (который будем брать в тест) 2021-10-02.

6. Есть g0 и g1, которые меньше 0. Непонятно, как они оказались в данных, так что тоже удалим их как вбросы.

7. Также по заданию нужно оставить только те строки, в которых banner_id = banner_id0

Ровно этим и будем заниматься в функции feature_engineering. Правда в OneHotEncoding отправим все данные, потому что, они так сожмутся. А без сжатия трудно закодировать даже один столбец.

In [9]:
from sklearn.preprocessing import OneHotEncoder

#feature engineering according to the analysis and spliting into train and test
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    #remove incorrect and unnecessary data
    data = data.drop(['oaid_hash', 'rate0', 'rate1', 'impressions'], axis=1)
    data = data[data['banner_id'] == data['banner_id0']]
    data = data[data['date_time'] > '2021-09-20']
    data = data.drop(['banner_id0'], axis=1)
    data = data[(data['g1'] > 0) & (data['g0'] > 0)]
    data = data.dropna()

    #get coeffs of normal distribution for the last day
    coeffs = data[['date_time', 'g0', 'g1', 'coeff_sum0',  'coeff_sum1']]
    coeffs = coeffs[coeffs['date_time'] >= '2021-10-02'].drop(['date_time'], axis=1)

    #fit encoder for linear classification task
    data = data.drop(['g0', 'g1', 'coeff_sum0',  'coeff_sum1'], axis=1)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc = enc.fit(data.drop(['clicks', 'date_time', 'banner_id1'], axis=1))

    #get x_train and y_train for linear classification
    cur = data[data['date_time'] < '2021-10-02']
    x_train = cur.drop(['clicks', 'date_time', 'banner_id1'], axis=1)
    x_train = enc.transform(x_train)
    y_train = cur['clicks']

    #get x_test0 with banner_id0 and y_test
    cur = data[data['date_time'] >= '2021-10-02']
    x_test0 = cur.drop(['clicks', 'date_time', 'banner_id1'], axis=1)
    x_test0 = enc.transform(x_test0)
    y_test = cur['clicks']

    #get x_test1 with banner_id1
    cur['banner_id'] = cur['banner_id1']
    x_test1 = cur.drop(['clicks', 'date_time', 'banner_id1'], axis=1)
    x_test1 = enc.transform(x_test1)
    
    return x_train, y_train, x_test0, x_test1, y_test, coeffs

In [ ]:
x_train, y_train, x_test0, x_test1, y_test, coeffs = feature_engineering(df)
df = []

Берем линейную модель из первой домашки. И на всякий случай считаем метрики.

In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=0.01, penalty='l2', fit_intercept=False)
model = model.fit(x_train, y_train)
y_pred0 = model.predict(x_test0)
y_pred1 = model.predict(x_test1)

In [13]:
from sklearn.metrics import roc_auc_score, log_loss
auc_metric = roc_auc_score(y_test, model.predict_proba(x_test0)[:, 1])
log_loss_metric = log_loss(y_test, model.predict_proba(x_test0))
print('Auc: {:.3f}, log_loss: {:.3f}'.format(auc_metric, log_loss_metric))

Auc: 0.791, log_loss: 0.134


Считаем coeff_sum0_new и coeff_sum1_new из предсказаний модели.

In [14]:
from scipy.special import logit

coeffs['coeff_sum0_new'] = logit(model.predict_proba(x_test0)[:, 1])
coeffs['coeff_sum1_new'] = logit(model.predict_proba(x_test1)[:, 1])

Считаем вероятность того, что одна нормальная величина больше, чем другая. Выкладки не привожу, но их можно найти например здесь:
https://stats.stackexchange.com/questions/50501/probability-of-one-random-variable-being-greater-than-another

In [16]:
from scipy.stats import norm
#sf = 1 - sdf
coeffs['pi_0'] = norm.sf((coeffs['coeff_sum1'] - coeffs['coeff_sum0']) / np.sqrt(coeffs['g0'] ** 2 + coeffs['g1'] ** 2))
coeffs['pi_1'] = norm.sf((coeffs['coeff_sum1_new'] - coeffs['coeff_sum0_new']) / np.sqrt(coeffs['g0'] ** 2 + coeffs['g1'] ** 2))

И считаем cips для lambda=10

In [19]:
lam = 10
cips = np.sum(y_test * np.clip(coeffs['pi_1'] / coeffs['pi_0'], a_min=None, a_max=lam)) / len(y_test)


In [21]:
print(cips)

0.07326100611835924


Получили cips 0.07